### 导入参数

    输入dataset 和 checkpoint地址

    你的数据集要通过我的制作函函数做成dataset4格式
    训练用什么参数 下面就要用什么参数 和best_model_path是对应的


In [1]:
best_model_path = './checkpoints/step7_Autoformer_custom_ftM_sl17_ll17_pl7_dm512_nh8_el1_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0/checkpoint.pth'
data_path = 'dataset4.csv'

In [2]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=False, default=0, help='status')
parser.add_argument('--model_id', type=str, required=False, default='test', help='model id')
parser.add_argument('--model', type=str, required=False, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=False, default='custom', help='dataset type')
parser.add_argument('--root_path', type=str, default='./dataset/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default= data_path, help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='3Z', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=17, help='input sequence length')
parser.add_argument('--label_len', type=int, default=17, help='start token length')
parser.add_argument('--pred_len', type=int, default=7, help='prediction sequence length')

# model define
parser.add_argument('--bucket_size', type=int, default=4, help='for Reformer')
parser.add_argument('--n_hashes', type=int, default=4, help='for Reformer')
parser.add_argument('--enc_in', type=int, default=17, help='encoder input size')
parser.add_argument('--dec_in', type=int, default=17, help='decoder input size')
parser.add_argument('--c_out', type=int, default=17, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=1, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=3, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in encoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=50, help='train epochs')
parser.add_argument('--batch_size', type=int, default=64, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')

args = parser.parse_args([])

args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.dvices = args.devices.replace(' ', '')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

print('Args in experiment:')
print(args)

Exp = Exp_Main
setting = '1'.format(
    args.model_id,
    args.model,
    args.data,
    args.features,
    args.seq_len,
    args.label_len,
    args.pred_len,
    args.d_model,
    args.n_heads,
    args.e_layers,
    args.d_layers,
    args.d_ff,
    args.factor,
    args.embed,
    args.distil,
    args.des, 0)

Args in experiment:
Namespace(is_training=0, model_id='test', model='Autoformer', data='custom', root_path='./dataset/', data_path='dataset4.csv', features='M', target='3Z', freq='h', checkpoints='./checkpoints/', seq_len=17, label_len=17, pred_len=7, bucket_size=4, n_hashes=4, enc_in=17, dec_in=17, c_out=17, d_model=512, n_heads=8, e_layers=1, d_layers=1, d_ff=2048, moving_avg=25, factor=3, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=50, batch_size=64, patience=3, learning_rate=0.0001, des='test', loss='mse', lradj='type1', use_amp=False, use_gpu=True, gpu=0, use_multi_gpu=False, devices='0,1,2,3')


In [3]:
exp = Exp(args)

Use GPU: cuda:0


### 载入模型

In [4]:
from models import Informer, Autoformer, Transformer, Reformer
model = Autoformer.Model(exp.args).float()
# if exp.args.use_multi_gpu and exp.args.use_gpu:
#     model = nn.DataParallel(model, device_ids=exp.args.device_ids)
# return model

### 制作预测数据集

In [5]:
import os
import numpy as np
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from utils.timefeatures import time_features
import warnings
shuffle_flag = False
drop_last = False
batch_size = args.batch_size
freq = args.freq

class Dataset_Predict(Dataset):
    def __init__(self, root_path, data_path='ETTh1.csv', size=None,
                 features='S', target='OT', scale=True, timeenc=0, freq='h'):
        # size [seq_len, label_len, pred_len]
        # info
        if size is None:
            self.seq_len = 24 * 4 * 4
            self.label_len = 24 * 4
            self.pred_len = 24 * 4
        else:
            self.seq_len = size[0]
            self.label_len = size[1]
            self.pred_len = size[2]

        self.features = features
        self.target = target
        self.scale = scale
        self.timeenc = timeenc
        self.freq = freq

        self.root_path = root_path
        self.data_path = data_path
        self.__read_data__()

    def __read_data__(self):
        self.scaler = StandardScaler()
        df_raw = pd.read_csv(os.path.join(self.root_path, self.data_path))

        cols = list(df_raw.columns)
        cols.remove(self.target)
        cols.remove('date')
        df_raw = df_raw[['date'] + cols + [self.target]]

        if self.features == 'M' or self.features == 'MS':
            cols_data = df_raw.columns[1:]
            df_data = df_raw[cols_data]
        elif self.features == 'S':
            df_data = df_raw[[self.target]]

        if self.scale:
            self.scaler.fit(df_data.values)
            data = self.scaler.transform(df_data.values)
        else:
            data = df_data.values

        df_stamp = df_raw[['date']]
        df_stamp['date'] = pd.to_datetime(df_stamp.date)

        if self.timeenc == 0:
            df_stamp['month'] = df_stamp.date.apply(lambda row: row.month, 1)
            df_stamp['day'] = df_stamp.date.apply(lambda row: row.day, 1)
            df_stamp['weekday'] = df_stamp.date.apply(lambda row: row.weekday(), 1)
            df_stamp['hour'] = df_stamp.date.apply(lambda row: row.hour, 1)
            data_stamp = df_stamp.drop(['date'], 1).values
        elif self.timeenc == 1:
            data_stamp = time_features(pd.to_datetime(df_stamp['date'].values), freq=self.freq)
            data_stamp = data_stamp.transpose(1, 0)

        self.data_x = data
        self.data_y = data
        self.data_stamp = data_stamp

    def __getitem__(self, index):
        s_begin = index
        s_end = s_begin + self.seq_len
        r_begin = s_end - self.label_len
        r_end = r_begin + self.label_len + self.pred_len

        seq_x = self.data_x[s_begin:s_end]
        seq_y = self.data_y[r_begin:r_end]
        seq_x_mark = self.data_stamp[s_begin:s_end]
        seq_y_mark = self.data_stamp[r_begin:r_end]

        return seq_x, seq_y, seq_x_mark, seq_y_mark

    def __len__(self):
        return len(self.data_x) - self.seq_len - self.pred_len + 1

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)
timeenc = 1
data_set = Dataset_Predict(
        root_path=args.root_path,
        data_path=args.data_path,
        size=[args.seq_len, args.label_len, args.pred_len],
        features=args.features,
        target=args.target,
        timeenc=timeenc,
        freq=freq
    )
print(len(data_set))
data_loader = DataLoader(
    data_set,
    batch_size=batch_size,
    shuffle=shuffle_flag,
    drop_last=drop_last)   

51460


### 模型预测

In [6]:
len(data_set)

51460

In [7]:
len(pd.read_csv('./dataset/dataset4.csv'))

51483

In [8]:
model.load_state_dict(torch.load(best_model_path))

preds = []

exp.model.eval()
with torch.no_grad():
    for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(data_loader):
        batch_x = batch_x.float().to(exp.device)
        batch_y = batch_y.float()
        batch_x_mark = batch_x_mark.float().to(exp.device)
        batch_y_mark = batch_y_mark.float().to(exp.device)

        # decoder input
        dec_inp = torch.zeros([batch_y.shape[0], exp.args.pred_len, batch_y.shape[2]]).float()
        dec_inp = torch.cat([batch_y[:, :exp.args.label_len, :], dec_inp], dim=1).float().to(exp.device)
        # encoder - decoder
        if exp.args.use_amp:
            with torch.cuda.amp.autocast():
                if exp.args.output_attention:
                    outputs = exp.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                else:
                    outputs = exp.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
        else:
            if exp.args.output_attention:
                outputs = exp.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
            else:
                outputs = exp.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
        pred = outputs.detach().cpu().numpy()  # .squeeze()
        preds.append(pred)

preds = np.concatenate(preds)
np.save('output/real_prediction.npy', preds)


In [9]:
preds.shape # 数据量 步长 全部特征

(51460, 7, 17)

In [10]:
predict = np.load('output/real_prediction.npy')
predict.shape

(51460, 7, 17)

In [11]:
import pandas as pd
pd.DataFrame(data_set.inverse_transform(predict[0]))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1867.630005,1754.284424,2131.489502,2204.463379,126.818932,78.221825,444.776459,9.590447,152.070572,48.021187,118.309006,24.665787,53.508526,47.071323,115.454460,29.816372,51.273945
1,1820.166260,1759.241333,2116.440186,2150.069580,139.756256,63.445892,448.493408,9.625212,67.892319,18.493792,116.154724,66.637848,58.194321,93.478958,101.036209,40.430058,63.770309
2,1768.241943,1764.496216,2150.354736,2366.576416,154.084061,47.765762,475.747955,9.424036,29.135431,0.035655,117.855492,90.855919,74.388741,91.197289,80.533333,47.321491,76.862526
3,1810.677368,1795.363281,2197.546143,2582.252686,146.740173,33.567905,487.216492,9.297119,-1.919225,-0.594861,109.263893,101.140068,93.549515,69.025604,46.563080,53.229156,86.029472
4,1901.875610,1828.349487,2218.892334,2692.936768,126.010674,30.213240,473.884705,9.339736,-2.562345,16.336626,94.215149,98.221489,100.274239,41.226452,23.886257,55.688766,82.355293
5,1989.760864,1849.516479,2224.554932,2733.939453,109.438805,28.556757,454.912933,9.402867,6.405650,30.451944,80.510559,88.094696,101.334503,21.046455,14.952130,57.992477,75.450562
6,2047.513306,1960.694824,2158.362549,2474.389404,56.007587,45.670795,426.111053,10.218106,50.332405,114.957573,58.141872,117.360703,25.903709,-18.520638,-7.738566,49.010010,32.263119


In [13]:
pd.DataFrame(data_set.inverse_transform(predict[0])).astype(int)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1867,1754,2131,2204,126,78,444,9,152,48,118,24,53,47,115,29,51
1,1820,1759,2116,2150,139,63,448,9,67,18,116,66,58,93,101,40,63
2,1768,1764,2150,2366,154,47,475,9,29,0,117,90,74,91,80,47,76
3,1810,1795,2197,2582,146,33,487,9,-1,0,109,101,93,69,46,53,86
4,1901,1828,2218,2692,126,30,473,9,-2,16,94,98,100,41,23,55,82
5,1989,1849,2224,2733,109,28,454,9,6,30,80,88,101,21,14,57,75
6,2047,1960,2158,2474,56,45,426,10,50,114,58,117,25,-18,-7,49,32
